In [2]:
import pandas as pd
from tqdm import tqdm

## Add Slogan column to Clusters Alarms

In [13]:
cluster_filtered = pd.read_parquet("real-time clusters/20230101-20240101_real_time_clusters_filtered.parquet")

In [3]:
adsl = pd.read_parquet("alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.parquet")
pdh = pd.read_parquet("alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.parquet")
sdh = pd.read_parquet("alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1.parquet")
ptn = pd.read_parquet("alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet")

In [2]:
mob = pd.read_parquet("alarms datasets/mob/20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.parquet")

In [10]:
mob = mob[["alarm_id", "std_probable_cause_no"]]
adsl = adsl[["alarm_id", "std_probable_cause_no"]]
pdh = pdh[["alarm_id", "std_probable_cause_no"]]
sdh = sdh[["alarm_id", "std_probable_cause_no"]]
ptn = ptn[["alarm_id", "std_probable_cause_no"]]

In [11]:
all_alarms = pd.concat([adsl, pdh, sdh, ptn, mob])

In [14]:
clusters_with_slogans = pd.merge(cluster_filtered, all_alarms, on="alarm_id", how="left")

In [16]:
clusters_with_slogans.to_parquet("real-time clusters/20230101-20240101_real_time_clusters_filtered_with_slogans.parquet")

## Create Bitmap

In [3]:
clusters_with_slogans = pd.read_parquet("real-time clusters/20230101-20240101_real_time_clusters_filtered_with_slogans.parquet")

In [4]:
clusters_with_slogans = clusters_with_slogans[["cluster_id2", "alarm_id", "std_probable_cause_no"]]

In [5]:
grouped_clusters = clusters_with_slogans.groupby("cluster_id2")
unique_slogans = clusters_with_slogans['std_probable_cause_no'].unique()

In [10]:
# Definisci una funzione per calcolare la presenza di ciascuno slogan
def calculate_presence(group):
    presence = {slogan: 0 for slogan in unique_slogans}
    for slogan in group['std_probable_cause_no']:
        presence[slogan] = 1
    return pd.Series(presence)

tqdm.pandas()

# Applica la funzione a ciascun gruppo e crea un nuovo dataframe con i risultati
result_df = grouped_clusters.progress_apply(calculate_presence)

100%|██████████| 864160/864160 [03:18<00:00, 4349.18it/s]


In [13]:
# Rinomina la colonna 'None' in 'None' come stringa
result_df = result_df.rename(columns={None: 'None'})

In [15]:
result_df.to_parquet("real-time clusters/result_df_gt_2.parquet")